In [1]:
import json
import pandas as pd
import string
import regex as re
import pickle
import math

from pathlib import Path
from thoth.lab import common
from thoth.lab import solver
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
#from pyonmttok import Tokenizer
from nltk.corpus import stopwords
from string import punctuation

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/skotak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
FILE_NAME = "thoth-solver-dataset-v1.0.zip"
#common.extract_zip_file(FILE_NAME)

In [4]:
solver_reports = solver.aggregate_solver_results()
#solver_reports = solver.aggregate_solver_results(is_local=False)

INFO:thoth.lab.common:Number of file retrieved is: 10001


In [5]:
solver_reports_extracted_data = []
solver_errors = []
for solver_document in solver_reports:
    solver_report_extracted_data = solver.extract_data_from_solver_metadata(solver_document["metadata"])
    for k, v in solver_document["result"].items():
        solver_report_extracted_data[k] = v
        if k == "errors" and v:
            errors = solver.extract_errors_from_solver_result(v)
            for error in errors:
                solver_errors.append(error)
    
    packages = solver.extract_tree_from_solver_result(solver_document["result"])
    solver_report_extracted_data["packages"] = packages
    solver_reports_extracted_data.append(solver_report_extracted_data)


In [6]:
solver_total_errors_df = pd.DataFrame(solver_errors).reset_index()

solver_total_errors_df.head(10)

,index,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout
0,0,tryton,3.2.20,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting tryton==3.2.20\n Downloading https...,60.0
1,1,pyobjc-framework-contacts,4.0b1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting pyobjc-framework-contacts==4.0b1\n ...,60.0
2,2,numpy,1.14.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting numpy==1.14.0\n Downloading https:...,60.0
3,3,hachoir-core,1.3,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting hachoir-core==1.3\n Downloading ht...,60.0
4,4,xattr,0.7.7,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting xattr==0.7.7\n Downloading https:/...,60.0
5,5,spacy,2.1.1.dev0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting spacy==2.1.1.dev0\n Downloading ht...,60.0
6,6,fastavro,0.16.5,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting fastavro==0.16.5\n Downloading htt...,60.0
7,7,pyeclib,1.6.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting pyeclib==1.6.0\n Downloading https...,60.0
8,8,happybase,0.8,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting happybase==0.8\n Downloading https...,60.0
9,9,mahotas,0.6.1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting mahotas==0.6.1\n Downloading https...,60.0


In [7]:
def split_log(log_messages):
    ids_with_different_log_pattern = []
    Error_info, command_info, cwd, Complete_output, ERROR, specific_error = {}, {}, {}, {}, {}, {}
    for idx, msg in enumerate(log_messages):
        sentences = [x.strip() for x in msg.split('\n')]
        for id, sent in enumerate(sentences):
            if re.match(r"^Command.*", sent):
                Error_info[idx] = sent
            elif re.match(r"^command.*", sent):
                if idx in command_info.keys() and sent != command_info[idx]:
                    command_info[idx].append(sent)
                else:
                    command_info[idx] = [sent]
            elif re.match(r"^cwd.*", sent):
                if idx in cwd.keys() and sent != cwd[idx]:
                    cwd[idx].append(sent)
                else:
                    cwd[idx] = [sent]
            elif re.match(r"^Complete output.*", sent):
                number_of_lines = re.findall(r'\d+', sent) 
                if idx in Complete_output.keys() and sent != Complete_output[idx]:
                    Complete_output[idx].append(sentences[id:id+int(number_of_lines[0])+1])
                else:
                    Complete_output[idx] = sentences[id:id+int(number_of_lines[0])+1]
            elif re.match(r".*Error.*", sent):
                if idx in specific_error.keys() and sent != specific_error[idx]:
                    specific_error[idx].append(sent)
                else:
                    specific_error[idx] = [sent]
            elif re.match(r"^ERROR.*", sent):
                if idx in ERROR.keys() and sent != ERROR[idx]:
                    ERROR[idx].append(sent)
                else:
                    ERROR[idx] = [sent]
                    
    return Error_info, command_info, cwd, Complete_output, ERROR, specific_error

print(len(solver_total_errors_df['message']))

1624


In [8]:
Error_info, command_info, cwd, Complete_output, ERROR, specific_error= split_log(solver_total_errors_df['message'])
print(len(Error_info), len(command_info), len(cwd), len(Complete_output), len(ERROR), len(specific_error))

1604 1233 1233 1233 1598 901


In [9]:
solver_total_errors_df['Error_info']= solver_total_errors_df['index'].map(Error_info)
solver_total_errors_df['command_info']= solver_total_errors_df['index'].map(command_info)
solver_total_errors_df['cwd']= solver_total_errors_df['index'].map(cwd)
solver_total_errors_df['Complete_output']= solver_total_errors_df['index'].map(Complete_output)
solver_total_errors_df['ERROR']= solver_total_errors_df['index'].map(ERROR)
solver_total_errors_df['specific_error']= solver_total_errors_df['index'].map(specific_error)

In [10]:
clustering_data = {}
for idx, error in enumerate(solver_total_errors_df['specific_error']):
    #print(error)
    clustering_data[idx] = error
    if type(error) != list:
        if pd.isnull(error):
            clustering_data[idx] = solver_total_errors_df.iloc[idx]['ERROR']
        if type(solver_total_errors_df.iloc[idx]['ERROR']) != list:
            if pd.isnull(solver_total_errors_df.iloc[idx]['ERROR']):
                clustering_data[idx] = [solver_total_errors_df.iloc[idx]['Error_info']]
                if type(solver_total_errors_df.iloc[idx]['Error_info']) != list:
                    if pd.isnull(solver_total_errors_df.iloc[idx]['Error_info']):
                        clustering_data[idx] = [solver_total_errors_df.iloc[idx]['message']]

In [12]:
solver_total_errors_df['clustering_data']= solver_total_errors_df['index'].map(clustering_data)

In [13]:
_line_number = r'(at line[:]*\s*\d+)'
_url = r'(http[s]|root|srm|file)*:(//|/)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
_filepath = "(/[a-zA-Z\./]*[\s]?)"
path_regex = re.compile(r'(\b\w+://)\S+(?=\s)')
file_regex = re.compile(r'(\b[f|F]ile( exists)?:?\s?)/\S+(?=\s)')
py_regex = re.compile(r'/?\b[-./_a-zA-Z0-9]+\.py\b')
long_regex = re.compile(r'[-./_a-zA-Z0-9]{25,}')

In [14]:
def remove_whitespaces(sentence):
    return " ".join(sentence.split())

def substitute_path(string):
    string = path_regex.sub(r'\1', string)
    string = py_regex.sub(r'*.py', string)
    string = file_regex.sub(r'\1', string)
    string = long_regex.sub(r'', string)
    return string

def cleaner(log_messages):
    clean_log = {}
    table = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    for key in log_messages:
        for item in log_messages[key]:
            item =  " ".join(item.split('.'))
            item = re.sub(_line_number, "at line *", item)
            item = re.sub(_url, " ", item)
            item = re.sub("\d+", " ", item)
            item = re.sub(r'\b\w\b', ' ', item)
            item = substitute_path(item)
            if key in clean_log.keys():
                clean_log[key] = clean_log[key] + ' ' +remove_whitespaces(item)
            else:
                clean_log[key] = remove_whitespaces(item.translate(table).strip())
    return clean_log

In [15]:
clean_clustering_data = cleaner(clustering_data)

In [16]:
def tokenization(log_messages):
    tokenized = []
    for item in log_messages.values():
        tokenized.append(TreebankWordTokenizer().tokenize(item))
    stop = list(punctuation) + ["``", "''", '""']
    cleaned_tokens = []
    for row in tokenized:
        #cleaned_tokens.append([i for i in row if i.lower() not in stop and not i.lower().isnumeric()])
        cleaned_tokens.append(list(filter(None, [i.strip(punctuation) 
                                                 for i in row 
                                                 if i.lower() not in stop
                                                 and not i.lower().isnumeric()])))
    return cleaned_tokens

In [17]:
clean_clustering_data = tokenization(clean_clustering_data)

In [18]:
solver_total_errors_df['clean_clustering_data'] = clean_clustering_data

In [19]:
solver_total_errors_df.head(10)

,index,package_name,package_version,index_url,type,command,message,return_code,stderr,stdout,timeout,Error_info,command_info,cwd,Complete_output,ERROR,specific_error,clustering_data,clean_clustering_data
0,0,tryton,3.2.20,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting tryton==3.2.20\n Downloading https...,60.0,Command exited with non-zero status code (1): ...,[command: /home/solver/venv/bin/python3 -c 'im...,[cwd: /tmp/pip-install-_gczw75i/tryton/],"[Complete output (6 lines):, Traceback (most r...",[ERROR: Command errored out with exit status 1...,[SyntaxError: Missing parentheses in call to '...,[SyntaxError: Missing parentheses in call to '...,"[SyntaxError, Missing, parentheses, in, call, ..."
1,1,pyobjc-framework-contacts,4.0b1,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting pyobjc-framework-contacts==4.0b1\n ...,60.0,Command exited with non-zero status code (1): ...,[command: /home/solver/venv/bin/python3 -c 'im...,[cwd: /tmp/pip-install-t57p8sf3/pyobjc-framewo...,"[Complete output (15 lines):, Traceback (most ...",[ERROR: Command errored out with exit status 1...,[FileNotFoundError: [Errno 2] No such file or ...,[FileNotFoundError: [Errno 2] No such file or ...,"[FileNotFoundError, Errno, No, such, file, or,..."
2,2,numpy,1.14.0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting numpy==1.14.0\n Downloading https:...,60.0,Command exited with non-zero status code (1): ...,[command: /home/solver/venv/bin/python3 -u -c ...,"[cwd: /tmp/pip-install-vul_9fm4/numpy/, cwd: /...","[Complete output (306 lines):, Running from nu...","[ERROR: Failed building wheel for numpy, ERROR...","[raise RuntimeError(""Broken toolchain: cannot ...","[raise RuntimeError(""Broken toolchain: cannot ...","[raise, RuntimeError, Broken, toolchain, can, ..."
3,3,hachoir-core,1.3,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting hachoir-core==1.3\n Downloading ht...,60.0,Command exited with non-zero status code (1): ...,[command: /home/solver/venv/bin/python3 -c 'im...,[cwd: /tmp/pip-install-b_cqxm9t/hachoir-core/],"[Complete output (6 lines):, Traceback (most r...",[ERROR: Command errored out with exit status 1...,[SyntaxError: invalid syntax],[SyntaxError: invalid syntax],"[SyntaxError, invalid, syntax]"
4,4,xattr,0.7.7,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit statu...,Collecting xattr==0.7.7\n Downloading https:/...,60.0,Command exited with non-zero status code (1): ...,[command: /home/solver/venv/bin/python3 -c 'im...,[cwd: /tmp/pip-install-x7gd4535/xattr/],"[Complete output (102 lines):, WARNING: The di...",[ERROR: Command errored out with exit status 1...,[distutils.errors.DistutilsExecError: command ...,[distutils.errors.DistutilsExecError: command ...,"[distutils, errors, DistutilsExecError, comman..."
5,5,spacy,2.1.1.dev0,https://pypi.org/simple,command_error,/home/solver/venv/bin/python3 -m pip install -...,Command exited with non-zero status code (1): ...,1.0,ERROR: Command errored out with exit status ...,Collecting spacy==2.1.1.dev0\n Downloading ht...,60.0,Command exited with non-zero status code (1): ...,[command: /home/solver/venv/bin/python3 /home/...,[cwd: /tmp/pip-install-43ioejjp/spacy],"[Complete output (727 lines):, running bdist_w...","[ERROR: Failed building wheel for spacy, ERROR...",NaN,"[ERROR: Faile

In [20]:
with open("solver-errors-clean-clustering-data.txt", "wb") as fp: 
    pickle.dump(solver_total_errors_df['clean_clustering_data'], fp)

In [21]:
solver_total_errors_df.to_csv('solver-error-clean-data.csv', index=False)